In [1]:
import numpy as np
import pandas as pd
from multiprocess import Pool

from benchmarking.multiprocessing.functions import evaluate_ar, evaluate_multiplier_iid, evaluate_ma
from bstrapping.synthetic_time_series.log_ma import LogMovingAverage
from bstrapping.synthetic_time_series.moving_average import MovingAverage

In [2]:
benchmark = []
means_of_variance = []
std_of_variance = []
coverage_probability = []

list_name_weights = ['AR',
                     'Multiplier',
                     'MA',
                     ]

In [3]:
sample_size = 10000
runs = 250

parameters = np.array([0.5 ** i for i in range(1, 3)])

In [4]:
#samples = [
#    LogMovingAverage(parameters=parameters, mu=0).generate_samples(sample_size)
#    for _ in range(runs)]
samples = [
    MovingAverage(parameters=parameters, mean=2).generate_samples(sample_size)
    for _ in range(runs)]

In [5]:
#phi = lambda x: np.log(x)
#derivative_phi = lambda x: 1 / x
phi = lambda x: np.exp(x)
derivative_phi = lambda x: np.exp(x)

In [6]:
#time_series = LogMovingAverage(parameters=parameters, mu=0)
time_series = MovingAverage(parameters=parameters, mean=2)
mean = phi(time_series.mean)
asymptotic_variance = time_series.asymptotic_variance*derivative_phi(time_series.mean)**2

## Benchmark bootstraps

In [7]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_ar = p.map(lambda sample: evaluate_ar(sample, alpha=0.1, mean=mean, phi=phi), samples)

result = np.array(evaluations_ar)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))


CPU times: user 65.7 ms, sys: 34.2 ms, total: 99.9 ms
Wall time: 1min 47s


In [8]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_multiplier_iid = p.map(lambda sample: evaluate_multiplier_iid(sample, alpha=0.1, mean=mean, phi=phi),
                                   samples)

result = np.array(evaluations_multiplier_iid)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3

CPU times: user 60.4 ms, sys: 36 ms, total: 96.4 ms
Wall time: 2.31 s


In [9]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_ma = p.map(lambda sample: evaluate_ma(sample, alpha=0.1, mean=mean, phi=phi), samples)

result = np.array(evaluations_ma)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3

CPU times: user 161 ms, sys: 98.5 ms, total: 259 ms
Wall time: 50min 50s


## Concatination result

In [10]:
benchmark = [asymptotic_variance,
             sample_size] + means_of_variance + std_of_variance + [1 - 0.1] + coverage_probability

In [11]:
benchmark = pd.DataFrame([benchmark], columns=pd.MultiIndex.from_tuples([
                                                                            ("mean", "Asymptotic variance"),
                                                                            ("Sample size", "")] +
                                                                        [("mean", name,) for name in
                                                                         list_name_weights] +
                                                                        [("std", name,) for name in list_name_weights] +
                                                                        [("In confidence interval", "Confidence level")]
                                                                        +
                                                                        [("In confidence interval", name,) for name in
                                                                         list_name_weights]
                                                                        )).set_index(["Sample size"])

In [12]:
benchmark

mean                                           std  \
            Asymptotic variance          AR Multiplier          MA         AR   
Sample size                                                                     
10000                167.206834  163.254197   71.68121  163.481708  22.899622   

                                  In confidence interval                    \
            Multiplier         MA       Confidence level     AR Multiplier   
Sample size                                                                  
10000         7.090124  19.747737                    0.9  0.904      0.728   

                    
                MA  
Sample size         
10000        0.908

In [13]:
benchmark.to_csv(f"./data/delta_method/benchmark_{sample_size}_exp.csv")
benchmark.to_pickle(f"./data/delta_method/benchmark_{sample_size}_exp.pkl")